In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/faspy/raw_data/main/trips_data.csv') 
df.head()

,City,Product Type,Trip or Order Status,Request Time,Begin Trip Time,Begin Trip Lat,Begin Trip Lng,Dropoff Time,Dropoff Lat,Dropoff Lng,Distance (miles),Fare Amount,Fare Currency
0,458,Black,COMPLETED,2021-01-13 22:06:46 +0000 UTC,2021-01-13 22:11:10 +0000 UTC,-23.599752,-46.715954,2021-01-13 22:29:13 +0000 UTC,-23.559359,-46.666542,4.84,29.63,BRL
1,458,VIP,COMPLETED,2021-01-13 20:21:05 +0000 UTC,2021-01-13 20:27:29 +0000 UTC,-23.559298,-46.666454,2021-01-13 20:54:50 +0000 UTC,-23.599585,-46.715717,5.31,20.86,BRL
2,458,VIP,COMPLETED,2021-01-03 00:23:22 +0000 UTC,2021-01-03 00:33:00 +0000 UTC,-23.626593,-46.659410,2021-01-03 00:50:56 +0000 UTC,-23.559273,-46.666595,5.90,34.23,BRL
3,458,VIP,COMPLETED,2020-12-11 23:16:33 +0000 UTC,2020-12-11 23:27:32 +0000 UTC,-23.559410,-46.666435,2020-12-12 00:32:47 +0000 UTC,-23.425755,-46.481190,19.74,58.77,BRL
4,458,VIP,COMPLETED,2020-12-11 00:35:46 +0000 UTC,2020-12-11 00:46:46 +0000 UTC,-23.584846,-46.663590,2020-12-11 00:57:57 +0000 UTC,-23.559261,-46.666510,2.54,12.63,BRL


In [3]:
df.isnull().sum()

City                     0
Product Type             3
Trip or Order Status     0
Request Time             0
Begin Trip Time          0
Begin Trip Lat          29
Begin Trip Lng          29
Dropoff Time             0
Dropoff Lat             29
Dropoff Lng             29
Distance (miles)         0
Fare Amount              0
Fare Currency            3
dtype: int64

In [4]:
df.drop(df[df['Product Type'] == 'UberEATS Marketplace'].index, inplace=True)
df.drop(df[df['Trip or Order Status'] != 'COMPLETED'].index, inplace=True)

In [5]:
# Categories reclassification
product_mapping = {'uberX':'UberX','uberx':'UberX','uberX VIP':'Uber VIP','VIP':'Uber VIP','POOL':'UberPool', 'Pool':'UberPool', 'Black':'UberBLACK',
                   'POOL: MATCHED':'UberPool', 'uberPOOL':'UberPool','uberPOOL: MATCHED':'UberPool','Pool: MATCHED':'UberPool'}
df['Product Type'].replace(product_mapping, inplace=True)

In [6]:
df['Request Time'] = pd.to_datetime(df['Request Time'].apply(lambda x: x.replace(' +0000 UTC', '')))
df['Begin Trip Time'] = pd.to_datetime(df['Begin Trip Time'].apply(lambda x: x.replace(' +0000 UTC', '')))
df['Dropoff Time'] = pd.to_datetime(df['Dropoff Time'].apply(lambda x: x.replace(' +0000 UTC', '')))

In [7]:
df['year'] = df['Request Time'].dt.year
df['month'] = df['Request Time'].dt.month
df['weekday'] = df['Request Time'].dt.weekday
df['time'] = df['Request Time'].dt.time

In [8]:
df[['Request Time','Begin Trip Time','Dropoff Time','year','month','weekday','time']].head()

,Request Time,Begin Trip Time,Dropoff Time,year,month,weekday,time
0,2021-01-13 22:06:46,2021-01-13 22:11:10,2021-01-13 22:29:13,2021,1,2,22:06:46
1,2021-01-13 20:21:05,2021-01-13 20:27:29,2021-01-13 20:54:50,2021,1,2,20:21:05
2,2021-01-03 00:23:22,2021-01-03 00:33:00,2021-01-03 00:50:56,2021,1,6,00:23:22
3,2020-12-11 23:16:33,2020-12-11 23:27:32,2020-12-12 00:32:47,2020,12,4,23:16:33
4,2020-12-11 00:35:46,2020-12-11 00:46:46,2020-12-11 00:57:57,2020,12,4,00:35:46


In [9]:
df['distance_km'] = round(df['Distance (miles)']*1.60934,2)
df['amount_km'] = round(df['Fare Amount']/df['Distance (miles)'],2)
df[['distance_km', 'amount_km']].head()

,distance_km,amount_km
0,7.79,6.12
1,8.55,3.93
2,9.50,5.80
3,31.77,2.98
4,4.09,4.97


In [10]:
df['request_lead_time'] = (df['Begin Trip Time'] - df['Request Time']).apply(lambda x: round(x.total_seconds()/60,1))
df['trip_duration'] = (df['Dropoff Time'] - df['Begin Trip Time']).apply(lambda x: round(x.total_seconds()/60,1))
df[['request_lead_time','trip_duration']].head()

,request_lead_time,trip_duration
0,4.4,18.1
1,6.4,27.4
2,9.6,17.9
3,11.0,65.2
4,11.0,11.2


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 444 entries, 0 to 551
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   City                  444 non-null    int64         
 1   Product Type          444 non-null    object        
 2   Trip or Order Status  444 non-null    object        
 3   Request Time          444 non-null    datetime64[ns]
 4   Begin Trip Time       444 non-null    datetime64[ns]
 5   Begin Trip Lat        444 non-null    float64       
 6   Begin Trip Lng        444 non-null    float64       
 7   Dropoff Time          444 non-null    datetime64[ns]
 8   Dropoff Lat           444 non-null    float64       
 9   Dropoff Lng           444 non-null    float64       
 10  Distance (miles)      444 non-null    float64       
 11  Fare Amount           444 non-null    float64       
 12  Fare Currency         444 non-null    object        
 13  year                